In [1]:
import os
import sys
import shutil
import argparse
import glob
import logging
import configparser
import logging.config
import tqdm
import subprocess
import datetime
from time import time
from multiprocessing import Pool, Queue
import psutil
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import csv
from PIL import Image
import os
import pandas as pd

In [2]:
def classify(model, input_dir, output_dir):
    images = []
    image_files = []
    for img in os.listdir(input_dir):
        if os.path.splitext(img)[1] == '.png':
            image_files.append(img)
            img = tf.keras.preprocessing.image.load_img(input_dir + '/' + img, target_size=(int(config['classification']['image_size']),int(config['classification']['image_size'])), color_mode='grayscale')
            img = np.expand_dims(img, axis=0)
            images.append(img)
    images = np.vstack(images)
    
    predictions = model.predict(images)
    prediction_labels = np.argmax(predictions, axis=-1)
    df = pd.DataFrame(predictions, index=image_files)
    df.to_csv(output_dir + '_' + 'prediction.csv', index=True, header=True, sep=',')
    

In [3]:
def pad(path, ratio=0.2):
    dirs = os.listdir(path)
    count = 0
    for item in dirs:
        if not os.path.isfile(path + '/' + item):
            dirs2 = os.listdir(path + '/' + item + "/")
            for item2 in dirs2:
                if os.path.isfile(path+'/'+item+"/"+item2):
                    if os.path.splitext(path+'/'+item+"/"+item2)[1] == '.png':
                        im = Image.open(path+'/'+item+"/"+item2)
                        width, height = im.size
                        if width > height * (1. + ratio):
                            left = 0
                            right = width
                            top = (width - height)//2
                            bottom = width
                            result = Image.new(im.mode, (right, bottom), (255))
                            result.paste(im, (left, top))
                            im.close()
                            result.save(path + '/'+item+"/"+item2)
                            count+=1
                        elif height > width * (1. + ratio):
                            left = (height - width)//2
                            right = height
                            top = 0
                            bottom = height
                            result = Image.new(im.mode, (right, bottom), (255))
                            result.paste(im, (left, top))
                            im.close()
                            result.save(path + '/'+item+"/"+item2)
                            count+=1
    count

In [4]:
directory = '../../analysis/camera0/segmentation/test1-REG/'
configuration = 'default.ini'

In [5]:
v_string = "V2023.11.13"
session_id = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")).replace(':', '')
print(f"Starting Plankline Classification Script {v_string}")

config = configparser.ConfigParser()
config.read(configuration)

Starting Plankline Classification Script V2023.11.13


['default.ini']

In [6]:
# Load model
model_path = '../../model/Beta/'
model = tf.keras.models.load_model(model_path)

In [7]:
segmentation_dir = os.path.abspath(directory)  # /media/plankline/Data/analysis/segmentation/Camera1/Transect1-reg
classification_dir = segmentation_dir.replace('segmentation', 'classification')  # /media/plankline/Data/analysis/segmentation/Camera1/Transect1-reg
classification_dir = classification_dir + '-' + config["classification"]["model_name"] # /media/plankline/Data/analysis/segmentation/Camera1/Transect1-reg-Plankton
fast_scratch = config['classification']['fast_scratch'] + "/classify-" + session_id
    
os.makedirs(classification_dir, int(config['general']['dir_permissions']), exist_ok = True)
os.makedirs(fast_scratch, int(config['general']['dir_permissions']), exist_ok = True)
    
root = os.listdir(segmentation_dir)

In [8]:
n_pad = pad(segmentation_dir)
print(f"Number of images padded: {n_pad}.")

Number of images padded: None.


In [9]:
for r in root:
    classify(model, segmentation_dir + '/' + r, classification_dir + '/' + r)

51/51 [==============================] - 2s 30ms/step
